In [84]:
#-*- coding:utf-8 -*-
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.utils.data as data

train = pd.read_csv("data.csv",parse_dates=["Date"])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wind1 = pd.read_csv("./dataset/wind1.csv")
wind2 = pd.read_csv("./dataset/wind2.csv")
wind3 = pd.read_csv("./dataset/wind3.csv")
wind4 = pd.read_csv("./dataset/wind4.csv")

In [85]:
from torch.optim.lr_scheduler import ExponentialLR

# Hyper-Parameters
sequence_length = 36
input_size = 2
hidden_size = 64
num_layers = 3
output_size = 4
batch_size = 64
num_epochs = 32
learning_rate = 1e-5

In [86]:
class mydata(data.Dataset):
    def __init__(self, input_dat, y,seq_length):
        self.input_dat = torch.tensor(input_dat).float()
        self.y = torch.tensor(y).float()
        self.seq_length = seq_length

    def __getitem__(self, idx):
        if idx >= self.seq_length - 1:
            i_start = idx - self.seq_length + 1
            x = self.input_dat[i_start:(idx-3),:]
            y = self.y[(idx-3):(idx+1)]
        elif idx>=3:
            padding = self.input_dat[0].repeat(self.seq_length-idx-1,1)
            x = self.input_dat[0:(idx-3),:]
            x = torch.cat((padding,x),0)
            y = self.y[(idx-3):(idx+1)]
        else:
            x = self.input_dat[0].repeat(self.seq_length-4,1)
            padding_y = self.y[0].repeat(3-idx)
            y = self.y[0:(idx+1)]
            y = torch.cat((padding_y,y),0)
        return x, y
    
    def __len__(self):
        return self.input_dat.shape[0]

Using two strategies,

1. Test using MA data and Test relative error using MA data
2. Test using original data and test re using original data

Later, I'll figure out how to test using original data

Split the 4th dataset to test set and the first 3 dataset as training set.

In [87]:
# Training Sets
train1 = np.array(wind1[["GridPowerMA","WindSpdMA"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPowerMA","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPowerMA","WindSpdMA"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Test Sets
test = np.array(wind4[["GridPowerMA","WindSpdMA"]][19:])
test_target = np.array(wind4["GridPower"][19:])
test_data = mydata(test, test_target, sequence_length)
test_data1 = mydata(test,test[:,1],sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [88]:
##Build RNN(LSTM)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

lstm_ma = RNN(input_size, hidden_size, num_layers, output_size).to(device)
lstm_o = RNN(input_size,hidden_size, num_layers, output_size).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
op_ma = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
op_o = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
scheduler_ma = ExponentialLR(op_ma,gamma=0.9)
scheduler_o = ExponentialLR(op_o,gamma=0.9)

In [89]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    scheduler_ma.step()

0.12503021955490112
0.07127312570810318
0.03707216680049896
0.11722496151924133
0.07402026653289795
0.03522725775837898
0.11703823506832123
0.0683155283331871
0.03375053405761719
0.10736389458179474
0.06292068958282471
0.0316426046192646
0.11460758000612259
0.05641068518161774
0.030776571482419968
0.0976174846291542
0.05728388577699661
0.03062061220407486
0.10358235239982605
0.06373921036720276
0.029408881440758705
0.09177130460739136
0.06415273994207382
0.028174199163913727
0.08733763545751572
0.05317838862538338
0.027636509388685226
0.08726654201745987
0.05572456866502762
0.027044255286455154
0.09670809656381607
0.05952068790793419
0.026229267939925194
0.11329503357410431
0.06489556282758713
0.025895781815052032
0.08331383019685745
0.05962013825774193
0.02497721277177334
0.08294116705656052
0.05752996355295181
0.024814685806632042
0.1025303304195404
0.04091200232505798
0.02418317273259163
0.1038050651550293
0.04930054396390915
0.023855863139033318
0.0776551142334938
0.050240635871887

In [90]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.1020)
0.22163539127887863
0.8823466300964355
0.12902525812387466
0.5421052631578948


In [91]:
# Evaluation using ma data
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat1:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.0521)
0.20058923657133915
0.4693530201911926
0.1804942712187767
0.32719298245614037


If just input the original data, let's see what will happen.

In [92]:
# Training Sets
train1 = np.array(wind1[["GridPower","WindSpd"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPower","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPower","WindSpd"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [93]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    scheduler_o.step()

0.154197096824646
0.10565153509378433
0.02195463888347149
0.15748901665210724
0.10910704731941223
0.021627314388751984
0.1565721035003662
0.11199020594358444
0.02053685672581196
0.16052286326885223
0.11380713433027267
0.02082657441496849
0.16602349281311035
0.09990153461694717
0.02042916975915432
0.13454173505306244
0.08956239372491837
0.019922997802495956
0.14994120597839355
0.1090225800871849
0.02024572528898716
0.11751076579093933
0.10660703480243683
0.020194604992866516
0.14819836616516113
0.11515503376722336
0.01932639628648758
0.16899263858795166
0.11235499382019043
0.01968166045844555
0.1738917976617813
0.09745821356773376
0.01966996304690838
0.1491565704345703
0.10136004537343979
0.019439492374658585
0.16803646087646484
0.09872017800807953
0.018883969634771347
0.14039810001850128
0.1152818500995636
0.018544567748904228
0.16711950302124023
0.10878886282444
0.01796245202422142
0.17332381010055542
0.1109936535358429
0.018340054899454117
0.1455560028553009
0.10605756938457489
0.017

In [94]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.0968)
0.21617527019820715
0.8615908622741699
0.12745513767004013
0.5605263157894737


In [95]:
import xgboost as xgb
print("xgb",xgb.__version__)

xgb 1.5.0
